In [4]:
# Need some pre-requisites

!pip install langchain
!pip install huggingface_hub
!pip install sentence_transformers
!pip install unstructured[local-inference] -q
!pip install -q faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.2/178.2 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=bae66a222ddc60a104a9ac1501bbe5b63786537d897bbe19c663a14a5b52166f
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 26.7 MB/s 

In [6]:
# FAISS needed to store the vector representations of PDF text data from UnstructuredPDFLoader
# QA will provide me with answer based data from PDF provided, to the prompt given

from langchain.document_loaders import UnstructuredPDFLoader
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain


In [7]:
# PLEASE PROVIDE HUGGINGFACE TOKEN, SO THAT WE ARE ABLE TO USE HUGGINGFACE MODELS

import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "PUT_YOUR_HUGGING_FACE_TOKEN_HERE"

In [8]:
# Load the document of where the data is.. In my case., Clothes Catalogue

loader = UnstructuredPDFLoader("/content/doc2.pdf")
document = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
# Take a document and split it into chunks of a specified size while considering overlap and using defined separators.
# Useful for processing large text documents in smaller, more manageable pieces.

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=0, separators=[" ", ",", "\n"])
docs = text_splitter.split_documents(document)

In [ ]:
# Numerical representations of words, sentences, or documents that capture semantic information are done with HuggingFaceEmbeddings
# Creating a FAISS index from the embedded documents, which allows for efficient similarity search based on the learned embeddings.

from langchain.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings()
db = FAISS.from_documents(docs, embedding)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
# Previously, By fetching Hugging Face Token, We use a HuggingFace LLM powered to fetch response to the qa_chain

from langchain import HuggingFaceHub

llm=HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha", model_kwargs={"temperature":0.2, "max_length":256})
chain = load_qa_chain(llm, chain_type="stuff")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [ ]:
# I provide a query, and based on the query and PDF's vector values, similarities are matched to fetch a response

query = "I am a Man, looking for white shirt in summer. Give me type, brand and colors. Suggest one more. Do not say anything other than that"
docs = db.similarity_search(query)

chain.run(input_documents=docs, question=query)

'\nType: Short-Sleeve Linen Tee\nBrand: Uniqlo\nColors: White\n\nAnother suggestion:\nType: Linen Button-Down Shirt\nBrand: J.Crew\nColors: White, Light Blue, Striped Patterns.'